# Missing value imputation

In [1]:
pip list | grep tensorflow

tensorflow-estimator          2.10.0
tensorflow-macos              2.10.0
tensorflow-metal              0.6.0
tensorflow-probability        0.18.0
Note: you may need to restart the kernel to use updated packages.


In [2]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/Users/wwhfreddy/miniforge3/envs/tensorflow_silicon/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15147757936544410311
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 6822276807893032896
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]Metal device set to: Apple M1 Pro



2022-11-20 18:51:24.255937: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-20 18:51:24.256222: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [57]:
import pandas as pd
import numpy as np

# Imputation without filling target variable

###LSTM Model

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras.callbacks import History, EarlyStopping
from tensorflow import keras

In [9]:
Data_dic_m = pd.ExcelFile('/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled.xlsx')
Data_dic_s = pd.ExcelFile('/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled.xlsx')

In [10]:
dic_country = \
{'AUS': 'Australia',
 'BRA': 'Brazil',
 'CAN': 'India',
 'CHN': 'China',
 'GBR': 'United Kingdom',
 'IND': 'India',
 'JPN': 'Japan',
 'SGP': 'Singapore',
 'USA': 'United States'}
pd.DataFrame(dic_country,index=[0]).T.reset_index().rename(columns = {'index': 'country_code', 0: 'country_name'})

,country_code,country_name
0,AUS,Australia
1,BRA,Brazil
2,CAN,India
3,CHN,China
4,GBR,United Kingdom
5,IND,India
6,JPN,Japan
7,SGP,Singapore
8,USA,United States


In [32]:
def get_sheet_by_methods(data, method_num, verbose = 0):
  sheet_names = data.sheet_names
  sheet_imputation_map = pd.DataFrame([[s, s[:3].strip(),s[3:]] for s in sheet_names], columns = ['sheet_name', 'country_code', 'imputation method'])
  methods = sheet_imputation_map['imputation method'].unique()
  if verbose:
    print('methods tried:',methods, len(methods))
  return list(sheet_imputation_map.loc[sheet_imputation_map['imputation method'] == methods[method_num]]['sheet_name'].values)

In [ ]:
class MRobustScaler():
      """
      Scale features using statistics that are robust to outliers.
      """
      def __init__(self, q1=0.2, q2=0.8):
          self.q1 = q1
          self.q2 = q2

      def fit(self, df, eps):
          """
          Return quantile range and median of all features

          Set qrange be 1 if feature has very small range, otherwise calculate
          the quantile range with specified quantile
          """
          df = df.astype('float64') # convert object to float
          self.df_median = df.median() #pd median uses np.nanmedian
          df_q1 = df.quantile(self.q1)
          df_q2 = df.quantile(self.q2)
          qrange = np.array(df_q2 - df_q1) #Near constant features have a very small range
          if np.isscalar(qrange):
                if qrange < eps:
                      qrange = 1.0
          elif isinstance(qrange, np.ndarray):
                constant_mask = qrange < np.ones_like(qrange)*eps
                qrange[constant_mask] = 1.0
          self.qrange = qrange
          self.df_qrange = pd.Series(index = self.df_median.index, data=qrange)
          return self.qrange, self.df_median
      
      def get_fitted(self):
        return self.df_median, self.qrange

      def transform(self, df, eps = 0.1):
          qrange, df_median = self.fit(df, eps = eps)
          return (df - df_median)/qrange

In [ ]:
def train_test_split(df, train_ratio = 0.7, val_ratio = 0.9, IW = 3):
  """
  By default: train:val:test = 0.7:0.2:0.1
  """
  n = len(df)
  train_df = df[0:int(n*train_ratio)]
  val_df = df[(int(n*train_ratio)-IW):int(n*val_ratio)]
  test_df = df[(int(n*val_ratio)-IW):]
  return train_df, val_df, test_df

In [31]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df, scaled_data,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.scaled_data = scaled_data

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])
  
  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    return inputs, labels
  
  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)
    ds = ds.map(self.split_window)
    return ds
  
  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)
  
  @property
  def all_data(self):
    return self.make_dataset(self.scaled_data)

In [30]:
def fetch_pred_real(rbscaler, model):
  med, qrg = rbscaler.get_fitted()
  target_median, qrange = med.values[-1], qrg[-1]
  y_pred = model.model.predict(model.data_window.test)
  y_pred = np.array(y_pred)*qrange+target_median
  y_real = np.array([y*qrange+target_median for _, y in model.data_window.test]).reshape(y_pred.shape)
  return y_pred, y_real

In [29]:
def MAE(y_pred, y_real, verbose = 1):
  E = y_pred-y_real
  return np.mean(np.abs(E), axis = 0), np.mean(np.abs(E))

def MSE(y_pred, y_real):
  E = y_pred-y_real
  return np.mean(np.square(E), axis = 0), np.mean(np.square(E))

def RMSE(y_pred, y_real):
  np.sqrt(MSE(y_pred, y_real)[0])
  return np.sqrt(MSE(y_pred, y_real)[0]), np.sqrt(MSE(y_pred, y_real)[1])

In [28]:
def fetch_all_pred_real(rbscaler, model):
  med, qrg = rbscaler.get_fitted()
  target_median, qrange = med.values[-1], qrg[-1]
  y_pred = model.model.predict(model.data_window.all_data)
  y_pred = np.array(y_pred)*qrange+target_median
  y = None
  for _, j in  model.data_window.all_data:
    if y is None:
      y = j*qrange+target_median
    else:
      y = np.concatenate((y, j*qrange+target_median))
  if type(y).__module__ != np.__name__:
    y = y.numpy()
  y_real = y.reshape(y_pred.shape)
  return y_pred, y_real

In [27]:
def pat_pred(y, data):
  """
  y could be y_real or y_pred
  """
  d = pd.DataFrame({})
  for i in range(y.shape[1]):
    d[f'gap_{i}'] =  [np.nan] * i + list(y[:, i]) + [np.nan] * (y.shape[1]-i-1)
  d.index = data[IW:].index
  return d

In [26]:
def forecast(rbscaler, scaled_data, model, name = 'Manufacturing_value added_%_of_GDP'):
  med, qrg = rbscaler.get_fitted()
  target_median, qrange = med.values[-1], qrg[-1]
  y = model.model.predict(np.expand_dims(scaled_data.iloc[-3:, :], axis = 0))
  df =  pd.DataFrame({name: (np.array(y)*qrange+target_median).tolist()[0]}, index =  [scaled_data.index[-1]+i for i in range(1, 6)])
  return df

In [ ]:
SHIFT = 5
LW = 5
IW = 3
MAX_EPOCHS = 50
bs = 32
verbose = False
lr = 0.001
patience = 50
LOSS = tf.keras.losses.MeanAbsoluteError()


NameError: ignored

# Rolling or Interpolate

In [135]:
# only Rolling

In [223]:
def Rolling_Statistics(df,alphaa):
    df = df.copy()[df.Year>1979]
    d_f=df.iloc[:,1:-1].ewm(alpha=alphaa,ignore_na=True).mean()
    d_f.insert(0,"Year",df.iloc[:,0])
    d_f.insert(df.shape[1]-1,df.columns[-1],df.iloc[:,-1])
    return d_f
def create_df_output_rolling(df, Country_code, k):
    if k ==1:
        output_path = '/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled_rolling.xlsx'
    elif k==0:
        output_path = "/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled_rolling.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for x in range(len(Country_code)):
          for y in range(1):
            if y == 1:
              df_temp=df[x]
              df_temp = df_temp.copy()[df_temp.Year>1979]
              d_f=df_temp.iloc[:,1:-1].ewm(halflife=4).mean()
              d_f.insert(0,"Year",df_temp.iloc[:,0])
              d_f.insert(df_temp.shape[1]-1,df_temp.columns[-1],df_temp.iloc[:,-1])
              df_temp.to_excel(writer, sheet_name=Country_code[x]+' ' + "default alpha" , index=False)
            elif y ==0:
              if k == 0:
                if x==0:
                  alph= 0.12
                elif x==1:
                  alph= 0.12
                elif x==2:
                  alph= 0.11
                elif x==3:
                  alph= 0.06
                elif x==4:
                  alph= 0.01
                elif x==5:
                  alph= 0.01
                elif x==6:
                  alph= 0.11
                elif x==7:
                  alph= 0.01
                elif x==8:
                  alph= 0.05
              elif k == 1:
                if x==0:
                  alph= 0.11
                elif x==1:
                  alph= 0.11
                elif x==2:
                  alph= 0.09
                elif x==3:
                  alph= 0.08
                elif x==4:
                  alph= 0.02             
                elif x==5:
                  alph= 0.03
                elif x==6:
                  alph= 0.07
                elif x==7:
                  alph= 0.01
                elif x==8:
                  alph= 0.11
              df_temp=df[x][df[x].Year>1980]
              df_temp = Rolling_Statistics(df[x],0.08) 
              #df_temp=df_temp.iloc[:,-1].ewm(alpha=alph,ignore_na=True).mean()
              
              #df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              #df_temp.iloc[:,-1].interpolate(method ='spline', limit_direction ='forward',order=2,inplace=True)
              #df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              df_temp.dropna(axis=1).to_excel(writer, sheet_name=Country_code[x] , index=False)
create_df_output_rolling(Manufacture_df, Country_code,  1)
create_df_output_rolling(Service_df, Country_code,  0)

In [2]:
#only interpolate

In [229]:
Country_code=["AUS", "BRA", "CAN", "CHN", "GBR", "IND", "JPN", "SGP", "USA"]
Manufacture = pd.ExcelFile('/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_all.xlsx')
Service = pd.ExcelFile("/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_all.xlsx")
Manufacture_df=[]
Service_df=[]
for x in Country_code:
    Manufacture_df.append(pd.read_excel(Manufacture, x).replace('\.+', np.nan, regex=True))
    Service_df.append(pd.read_excel(Service, x).replace('\.+', np.nan, regex=True))

In [205]:
Manufacture_df[0]

,Year,Employment_in_industry_male,High_technology_exports,Research_development_expenditure,Researchers in R&D,Population_labor_rate,Population_density,Foreign_direct_investment,Net_trade_in_goods_services,Air_freight_million_ton_km,Container_port_traffic_TEU,Railways_goods_trans_million_ton-km,Industry_value_added_current_USD,GDP_per_capita_current_USD,Final_consumption,New_business_density,Employment_in_industry_%_of_total_employment,Manufacturing_value added_%_of_GDP
0,1990,NaN,NaN,NaN,NaN,66.851833,2.221353,2.716689,-3.259470e+09,1222.300049,NaN,NaN,8.961370e+10,18243.471418,2.255137e+11,NaN,NaN,13.793444
1,1991,31.709999,NaN,NaN,NaN,66.860327,2.249847,0.801486,9.560623e+08,1222.599976,NaN,NaN,9.012950e+10,18855.761996,2.458261e+11,NaN,23.510000,12.569810
2,1992,31.709999,NaN,NaN,NaN,66.781131,2.277313,1.518343,-5.959716e+08,1361.500000,NaN,NaN,8.745366e+10,18604.188270,2.539214e+11,NaN,23.290001,12.580159
3,1993,31.670000,NaN,NaN,NaN,66.662599,2.299702,1.702015,-1.312259e+09,1525.800049,NaN,NaN,8.372119e+10,17667.187100,2.409823e+11,NaN,23.299999,12.873184
4,1994,31.719999,NaN,NaN,NaN,66.582205,2.324174,1.381160,-4.713416e+09,1645.500000,NaN,NaN,8.644760e+10,18079.380179,2.456195e+11,NaN,23.170000,13.205020
5,1995,31.219999,NaN,NaN,NaN,66.572593,2.352420,3.606498,-5.240428e+09,1737.500000,NaN,106195.0,9.723938e+10,20358.333356,2.794402e+11,NaN,22.629999,13.123677
6,1996,30.610001,NaN,1.66323,3355.864990,66.555719,2.383531,1.137889,-5.332674e+08,1833.699951,NaN,110255.0,1.041882e+11,21904.294118,3.034181e+11,NaN,22.129999,12.742416
7,1997,30.320000,NaN,NaN,NaN,66.598197,2.410346,1.857942,1.646369e+09,1953.800049,NaN,119619.0,1.097684e+11,23509.423468,3.258452e+11,NaN,21.850000,12.303665
8,1998,30.129999,NaN,1.51453,3382.171875,66.676387,2.435599,1.902235,-6.467570e+09,1904.400024,NaN,125587.0,1.013654e+11,21345.970987,3.004165e+11,NaN,21.510000,12.470748
9,1999,29.760000,NaN,NaN,NaN,66.748195,2.463585,0.568215,-1.008243e+10,1693.000000,NaN,127963.0,9.584534e+10,20558.960402,2.967252e+11,NaN,21.160000,11.952832


In [230]:
def create_df_output_interpolate(df, Country_code, k):
    if k ==1:
        output_path = '/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled_inter.xlsx'
    elif k==0:
        output_path = "/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled_inter.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for x in range(len(Country_code)):
          for y in range(1):
            if y ==0:
              df_temp=df[x][df[x].Year>1979]
              #df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              for g in range(len(df_temp.columns)-1):
                  df_temp.iloc[:,g+1].interpolate(method ='spline', limit_direction ='both',order=1,inplace=True)
                    ]
              
              df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              df_temp.to_excel(writer, sheet_name=Country_code[x] , index=False)
create_df_output_interpolate(Manufacture_df, Country_code,  1)
create_df_output_interpolate(Service_df, Country_code,  0)

/Users/wwhfreddy/miniforge3/envs/tensorflow_silicon/lib/python3.9/site-packages/scipy/interpolate/fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/Users/wwhfreddy/miniforge3/envs/tensorflow_silicon/lib/python3.9/site-packages/scipy/interpolate/fitpack2.py:280: UserWarning: 
A theoretically impossible result was found during the iteration
process for finding a smoothing spline with fp = s: s too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


In [96]:
def Rolling_Statistics(df,alphaa):
    df = df.copy()[df.Year>1979]
    d_f=df.iloc[:,1:-1].ewm(alpha=alphaa,ignore_na=True).mean()
    d_f.insert(0,"Year",df.iloc[:,0])
    d_f.insert(df.shape[1]-1,df.columns[-1],df.iloc[:,-1])
    return d_f
def create_df_output(df, Country_code, k):
    if k ==1:
        output_path = '/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled.xlsx'
    elif k==0:
        output_path = "/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for x in range(len(Country_code)):
          for y in range(1):
            if y == 1:
              df_temp=df[x]
              df_temp = df_temp.copy()[df_temp.Year>1979]
              d_f=df_temp.iloc[:,1:-1].ewm(halflife=4).mean()
              d_f.insert(0,"Year",df_temp.iloc[:,0])
              d_f.insert(df_temp.shape[1]-1,df_temp.columns[-1],df_temp.iloc[:,-1])
              df_temp.to_excel(writer, sheet_name=Country_code[x]+' ' + "default alpha" , index=False)
            elif y ==0:
              if k == 0:
                if x==0:
                  alph= 0.12
                elif x==1:
                  alph= 0.12
                elif x==2:
                  alph= 0.11
                elif x==3:
                  alph= 0.06
                elif x==4:
                  alph= 0.01
                elif x==5:
                  alph= 0.01
                elif x==6:
                  alph= 0.11
                elif x==7:
                  alph= 0.01
                elif x==8:
                  alph= 0.05
              elif k == 1:
                if x==0:
                  alph= 0.11
                elif x==1:
                  alph= 0.11
                elif x==2:
                  alph= 0.09
                elif x==3:
                  alph= 0.08
                elif x==4:
                  alph= 0.02             
                elif x==5:
                  alph= 0.03
                elif x==6:
                  alph= 0.07
                elif x==7:
                  alph= 0.01
                elif x==8:
                  alph= 0.11
              df_temp = Rolling_Statistics(df[x],alph) 
              
              #df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              df_temp.iloc[:,-1].interpolate(method ='spline', limit_direction ='forward',order=2,inplace=True)
              df_temp = df_temp[df_temp[df_temp.columns[-1]].notna()]
              print(df_temp)
              df_temp.dropna(axis=1).to_excel(writer, sheet_name=Country_code[x] , index=False)
create_df_output(Manufacture_df, Country_code,  1)
create_df_output(Service_df, Country_code,  0)

    Year  Employment_in_industry_male  Research_development_expenditure  \
30  1990                          NaN                               NaN   
31  1991                    31.709999                               NaN   
32  1992                    31.709999                               NaN   
33  1993                    31.695086                               NaN   
34  1994                    31.702441                               NaN   
35  1995                    31.582266                               NaN   
36  1996                    31.369651                          1.663230   
37  1997                    31.162615                          1.663230   
38  1998                    30.975282                          1.584553   
39  1999                    30.769506                          1.584553   
40  2000                    30.721632                          1.580708   
41  2001                    30.549338                          1.580708   
42  2002                 

In [79]:
Data_dic_m = pd.ExcelFile('/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled.xlsx')
Data_dic_s = pd.ExcelFile('/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled.xlsx')
m_dfs = {}    
    
from scipy import interpolate

m_dfs = {}
for sheet in dic_country.keys():
    df = pd.read_excel(Data_dic_m, sheet_name=sheet)
    df['Year'] = [i.year for i in pd.to_datetime(df.Year, format='%Y')]
    df = df.set_index('Year')
    tmp_year = pd.DataFrame({'Year': [i for i in range(df.index[0], 2022)]})
    tmp_year['Year'] = [i.year for i in pd.to_datetime(tmp_year.Year, format='%Y')]
    df1 = df.join(tmp_year.set_index('Year'), on = 'Year', how = 'right')
    if 'Year' in df1.columns:
      df1 = df1.set_index('Year')
    tmp = {}
    for col in df.columns:
      s = interpolate.InterpolatedUnivariateSpline(x=df.index, y=df[col])
      ynew = s(df1.index)
      tmp[col] = ynew
    final_df = pd.DataFrame(tmp)
    final_df.index = df1.index
    m_dfs[sheet] = final_df
s_dfs = {}
for sheet in dic_country.keys():
    df = pd.read_excel(Data_dic_s, sheet_name=sheet)
    df['Year'] = [i.year for i in pd.to_datetime(df.Year, format='%Y')]
    df = df.set_index('Year')
    tmp_year = pd.DataFrame({'Year': [i for i in range(df.index[0], 2022)]})
    tmp_year['Year'] = [i.year for i in pd.to_datetime(tmp_year.Year, format='%Y')]
    df1 = df.join(tmp_year.set_index('Year'), on = 'Year', how = 'right')
    if 'Year' in df1.columns:
      df1 = df1.set_index('Year')
    tmp = {}
    for col in df.columns:
      s = interpolate.InterpolatedUnivariateSpline(x=df.index, y=df[col])
      ynew = s(df1.index)
      tmp[col] = ynew
    final_df = pd.DataFrame(tmp)
    final_df.index = df1.index
    s_dfs[sheet] = final_df
for country in dic_country:
    k=s_dfs[country].index
    g=m_dfs[country].index
    s_dfs[country].insert(0, 'Year', k)
    m_dfs[country].insert(0, 'Year', g)

m_dfs["USA"]

,Year,Employment_in_industry_male,Research_development_expenditure,Researchers in R&D,Population_labor_rate,Population_density,Foreign_direct_investment,Air_freight_million_ton_km,Container_port_traffic_TEU,Industry_value_added_current_USD,GDP_per_capita_current_USD,Final_consumption,New_business_density,Employment_in_industry_%_of_total_employment,Manufacturing_value added_%_of_GDP
Year,,,,,,,,,,,,,,,
1980,1980,NaN,NaN,NaN,65.770297,24.809039,0.585866,8615.400391,NaN,NaN,12574.791506,2.204317e+12,NaN,NaN,NaN
1981,1981,NaN,NaN,NaN,65.872147,24.938499,0.699555,8610.903036,NaN,NaN,13316.227425,2.329542e+12,NaN,NaN,NaN
1982,1982,NaN,NaN,NaN,65.960715,25.070933,0.675452,8493.419262,NaN,NaN,13732.901040,2.439994e+12,NaN,NaN,NaN
1983,1983,NaN,NaN,NaN,66.032696,25.205300,0.569461,8726.978347,NaN,NaN,14267.579453,2.569015e+12,NaN,NaN,NaN
1984,1984,NaN,NaN,NaN,66.087944,25.340488,0.583264,9117.095326,NaN,NaN,14978.415706,2.708025e+12,NaN,NaN,NaN
1985,1985,NaN,NaN,NaN,66.127262,25.479721,0.504250,9238.485514,NaN,NaN,15690.964396,2.860105e+12,NaN,NaN,NaN
1986,1986,NaN,NaN,NaN,66.134312,25.625414,0.538074,9510.782858,NaN,NaN,16357.698856,3.010831e+12,NaN,NaN,NaN
1987,1987,NaN,NaN,NaN,66.118545,25.775691,0.676737,9966.575642,NaN,NaN,17025.535217,3.163645e+12,NaN,NaN,NaN
1988,1988,NaN,NaN,NaN,66.085838,25.931362,0.746172,10628.465698,NaN,NaN,17769.115914,3.330737e+12,NaN,NaN,NaN


In [59]:
def create_df_output_final(df, dic_country, k):
    if k ==1:
        output_path = '/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Manufacturing_filled_1.xlsx'
    elif k==0:
        output_path = "/Users/wwhfreddy/Desktop/Accenture4-Evaluating-the-Attractiveness-of-a-Country/Data/cleaned/Service_filled_1.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for country in dic_country:
            df[country].dropna(axis = 1).to_excel(writer, sheet_name=country , index=False)
create_df_output_final(m_dfs, Country_code,  1)
create_df_output_final(s_dfs, Country_code,  0)
dic_country

{'AUS': 'Australia',
 'BRA': 'Brazil',
 'CAN': 'India',
 'CHN': 'China',
 'GBR': 'United Kingdom',
 'IND': 'India',
 'JPN': 'Japan',
 'SGP': 'Singapore',
 'USA': 'United States'}

# Tuning **hyperparamenters**

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
class LSTM_base:

  def __init__(self, lookback_width, num_features, loss=None, layer1=512, layer2=256, layer3=256, label_width=1):
    self.lookback_width = lookback_width
    self.num_features = num_features
    self.loss = loss 
    self.layer1 = layer1
    self.layer2 = layer2
    self.layer3 = layer3
    self.label_width = label_width
    self.create_model()
    self.history = None

  def create_model(self):
    input = layers.Input(shape=(self.lookback_width, self.num_features), name='input')
    x = layers.LSTM(self.layer1, return_sequences=True)(input)
    x = layers.Dropout(0.05)(x)
    x = layers.LSTM(self.layer2, return_sequences=True)(x)
    x = layers.Dropout(0.1)(x)
    x = layers.LSTM(self.layer3)(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(self.label_width)(x)
    model = keras.Model(input, outputs=output)
    self.model = model

  def compile_and_fit(self, data_window, epochs, batch_size, verbose, learning_rate, patience=4):
    loss=self.loss,
    history = History()
    batch_size = batch_size[0] if isinstance(batch_size, tuple) else batch_size
    self.model.compile(loss=loss,
                       optimizer=keras.optimizers.Adam(learning_rate=learning_rate))
    self.history = self.model.fit(data_window.train,
                                  epochs=epochs,
                                  batch_size=batch_size,
                                  validation_data=data_window.val, 
                                  callbacks=[history, EarlyStopping(monitor='val_loss',
                                                           patience=patience,
                                                           mode='auto',
                                                           restore_best_weights=True)],
                                  verbose=verbose,
                                  shuffle=False)
    return self.history

  def plot(self):
    display(tf.keras.utils.plot_model(self.model, show_shapes=True))
  
  def train(self, data_window, MAX_EPOCHS, bs, verbose, lr, patience):
    start = time()
    self.data_window = data_window
    self.compile_and_fit(data_window=data_window,
                                epochs=MAX_EPOCHS,
                                batch_size=bs,
                                verbose=verbose,
                                learning_rate=lr,
                                patience=patience)
    delta=time()-start
    self.delta = delta
    print('Total Modelling Time: ',delta)

In [ ]:
SHIFT = 5
LW = 5
IW = 3
MAX_EPOCHS = 50
bs = 32
verbose = False
lr = 0.001
patience = 50
LOSS = tf.keras.losses.MeanAbsoluteError()

## Grid Search

In [ ]:
import itertools

lr = [0.001,0.005,0.01,0.015]
bs = [16,32,64,128]
patience = [30,40,50,70]
MAX_EPOCHS = [25,50,75,100]
LOSS=tf.keras.losses.MeanAbsoluteError()

hyper_para = (lr, bs, patience, MAX_EPOCHS)
hyper_para_grid = tuple(itertools.product(*hyper_para, repeat=1))
print(len(hyper_para_grid))
for lr, bs, patience, MAX_EPOCHS in hyper_para_grid:
  print('lr',lr, 'Batch size', bs, 'patience', patience,'MAX_EPOCHS', MAX_EPOCHS)

256
lr 0.001 Batch size 16 patience 30 MAX_EPOCHS 25
lr 0.001 Batch size 16 patience 30 MAX_EPOCHS 50
lr 0.001 Batch size 16 patience 30 MAX_EPOCHS 75
lr 0.001 Batch size 16 patience 30 MAX_EPOCHS 100
lr 0.001 Batch size 16 patience 40 MAX_EPOCHS 25
lr 0.001 Batch size 16 patience 40 MAX_EPOCHS 50
lr 0.001 Batch size 16 patience 40 MAX_EPOCHS 75
lr 0.001 Batch size 16 patience 40 MAX_EPOCHS 100
lr 0.001 Batch size 16 patience 50 MAX_EPOCHS 25
lr 0.001 Batch size 16 patience 50 MAX_EPOCHS 50
lr 0.001 Batch size 16 patience 50 MAX_EPOCHS 75
lr 0.001 Batch size 16 patience 50 MAX_EPOCHS 100
lr 0.001 Batch size 16 patience 70 MAX_EPOCHS 25
lr 0.001 Batch size 16 patience 70 MAX_EPOCHS 50
lr 0.001 Batch size 16 patience 70 MAX_EPOCHS 75
lr 0.001 Batch size 16 patience 70 MAX_EPOCHS 100
lr 0.001 Batch size 32 patience 30 MAX_EPOCHS 25
lr 0.001 Batch size 32 patience 30 MAX_EPOCHS 50
lr 0.001 Batch size 32 patience 30 MAX_EPOCHS 75
lr 0.001 Batch size 32 patience 30 MAX_EPOCHS 100
lr 0.001 Ba

## PIPELINE

In [ ]:
int_value=0
country_valloss = \
{'AUS': int_value,
 'BRA': int_value,
 'CAN': int_value,
 'CHN': int_value,
 'GBR': int_value,
 'IND': int_value,
 'JPN': int_value,
 'SGP': int_value,
 'USA': int_value,}

In [ ]:
Best_lr=[]
Best_bs=[]
Best_patience=[]
Best_max_epochs=[]
Mean_Val_lost=[]
Dataset=[]

In [ ]:
def tuning_pipeline(hyper_parameter,dataset):
  min_val_loss = 999
  for lr, bs, patience, MAX_EPOCHS in hyper_parameter:
    for country in Country_code:
      method="optimized"
      if dataset == 0:
         data = m_dfs[(country, method)]
      elif dataset == 1:
         data = s_dfs[(country, method)]
  # drop all column 0
      data = data.dropna(axis=1, how='all')
  # drop nan row according to target
      data = data[data[data.columns[-1]].notna()]
      data = data.fillna(data.median())
      rbscaler = MRobustScaler()
      scaled_data = rbscaler.transform(data, 0.1)
      train_df, val_df, test_df=train_test_split(scaled_data, IW = IW + SHIFT)
      wg = WindowGenerator(input_width=IW, label_width=LW, shift=SHIFT,
                      train_df = train_df, val_df = val_df, test_df = test_df, 
                      scaled_data = scaled_data,
                      label_columns=[scaled_data.columns[-1]])
      model = LSTM_base(lookback_width=IW, num_features = len(scaled_data.columns),label_width=LW, loss = LOSS)
      lstm_model=model.train(data_window=wg, MAX_EPOCHS = MAX_EPOCHS, bs=bs, verbose=verbose, lr=lr, patience = patience)
      country_valloss[country] = model.history.history['val_loss'][-1]
    mean_val_loss=np.mean(list(country_valloss.values()))
    print(mean_val_loss)
    if mean_val_loss < min_val_loss:
       min_val_loss = mean_val_loss
       best_params = [lr, bs, patience, MAX_EPOCHS]
    Best_lr.append(lr)
    Best_bs.append(bs)
    Best_patience.append(patience)
    Best_max_epochs.append(MAX_EPOCHS)
    Mean_Val_lost.append(mean_val_loss)
    Dataset.append(datasett)
    print('best_params',best_params)

In [ ]:
tuning_pipeline(hyper_para_grid,1)

Total Modelling Time:  36.33246040344238
Total Modelling Time:  6.641213655471802
Total Modelling Time:  5.2954630851745605
Total Modelling Time:  6.000452995300293
Total Modelling Time:  5.581642389297485
Total Modelling Time:  5.17253565788269
Total Modelling Time:  6.271773338317871
Total Modelling Time:  5.32807731628418


KeyboardInterrupt: ignored

In [ ]:
tuning_pipeline(hyper_para_grid,2)